In [25]:
import os as os
import slippi as slp
from joblib import Parallel, delayed, dump, load
import time
import pickle   
# import _pickle as cpkl
import gzip


In [26]:
dataset_path = '../Slippi_Public_Dataset_v3/'
dump_folder = 'C:/Users/jaspa/Grant ML/game_dump'
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]


In [27]:
def process_file(slp_file, dataset_path):
    file_path = os.path.join(dataset_path, slp_file)
    load_t1 = time.time()
    game = slp.Game(file_path)
    load_time = time.time() - load_t1
    
    # Remove the '.slp' extension from the slp_file to get the base filename
    base_filename = slp_file.rsplit('.', 1)[0]
    
    # Construct the full path for the dump file with the .pkl extension
    dump_path = os.path.join(dump_folder, base_filename + '.pkl')

    # Dumping data with pickle using a high protocol
    # with gzip.open(dump_path, 'wb') as f:
    with open(dump_path, 'wb') as f:
        pickle.dump(game, f, protocol=pickle.HIGHEST_PROTOCOL)

    return load_time

In [28]:
num_files = 10
sum_load_time = 0.0


for slp_file in slp_files[:num_files]:
    sum_load_time += process_file(slp_file,dataset_path)

print(sum_load_time)

1.6322851181030273


In [29]:
dump_files = [file for file in os.listdir(dump_folder)]

sum_dump_load_time = 0.0

for dump_file in dump_files:
    dump_path = os.path.join(dump_folder, dump_file)  # Change the extension to .joblib for the dump file
    # Loading data with pickle
    t1 = time.time()
    # with gzip.open(dump_path, 'rb') as f:
    with open(dump_path, 'rb') as f:
        data = pickle.load(f)
    sum_dump_load_time += time.time()-t1

print(sum_dump_load_time)

1.018219232559204


In [30]:
# games = []


# for slp_file in slp_files[:num_files]:
#     file_path = os.path.join(dataset_path, slp_file)
#     games.append(slp.Game(file_path))
    
# # Construct the full path for the dump file with the .pkl extension
# dump_path = os.path.join(dump_folder, 'games_list' + '.pkl')

# # Dumping data with pickle using a high protocol
# with open(dump_path, 'wb') as f:
#     pickle.dump(games, f, protocol=pickle.HIGHEST_PROTOCOL)
    

In [31]:
# with open(dump_path, 'rb') as f:
#     data = pickle.load(f)

In [34]:
file = 'C:/Users/jaspa/Grant ML/master-diamond-9a7a67e8349465790063d59c.slp'
slp.Game(file)

ParseError: Parse error (C:\Users\jaspa\Grant ML\master-diamond-9a7a67e8349465790063d59c.slp @0x1ee0760): [Errno None] None: 'C:\\Users\\jaspa\\Grant ML\\master-diamond-9a7a67e8349465790063d59c.slp'

In [35]:
from slippi.game import Game as BaseGame
from slippi.parse import parse, ParseEvent

class CustomGame(BaseGame):
    def __init__(self, input, skip_frames: bool = False, parse_metadata: bool = True):
        self.start = None
        self.frames = []
        self.end = None
        self.metadata = None
        self.metadata_raw = None

        # Define custom parse events, excluding metadata events if parse_metadata is False
        parse_events = {
            ParseEvent.START: lambda x: setattr(self, 'start', x),
            ParseEvent.FRAME: self._add_frame,
            ParseEvent.END: lambda x: setattr(self, 'end', x)
        }
        
        if parse_metadata:
            parse_events[ParseEvent.METADATA] = lambda x: setattr(self, 'metadata', x)
            parse_events[ParseEvent.METADATA_RAW] = lambda x: setattr(self, 'metadata_raw', x)

        parse(input, parse_events, skip_frames)

